In [1]:

import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [3]:
anss = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=5,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_depth=True
)

Generating the catalog...
Downloading...
train


100%|██████████| 3/3 [00:00<00:00, 83.54it/s]


val


100%|██████████| 1/1 [00:00<00:00, 242.64it/s]


test


100%|██████████| 1/1 [00:00<00:00, 263.48it/s]


Success!
Catalog saved to /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog


In [5]:
anss.train[1].extra_feat

tensor([ 93000.,  40400.,  48800.,  37300.,  86700., 206100.,  30900.,  37800.,
         18300.,  44500.,  38300.,  31900.,  55100.,  40300.,  66800.,  12000.,
         53000.,  52000.,  78900.,  70700.,  10000.,  45900.,  42800.,  55000.,
         28800.,  29900.,  35000.,  31100.,  66700.,  88500.,  37700.,  52500.,
         36200.,  35000.,  10000.,  35000.,  27100.,  64800.,  24600.,  47000.,
         31200.,  35000.,  83200.,  35000.,  35000.,  46400.,  26100.,  48400.,
         42500.,  58900.,  29400.,   5600.,  31500.,   6900.,  30900.,  10900.,
         31000.,  31700.,  48300.,  67700.,  20600.,  14000.,  15600.,  40800.,
         35000.,  22500.,  30100.,  40100.,  33700.,  38400.,  35000.,  34300.,
         80400.,  61300.,  41500.,  35000.,  47900.,  56800.,  35000.,  47100.,
         37100.,   8800.,  87000.,  10000.,  32200.,  69800.,  35100.,  31200.,
         20500., 137800.,  23400.,   7900.,  34600.,  27400.,  74700.,  44000.,
         35000.,  28100.,  27600.,  5790

In [6]:

dl_train = anss.train.get_dataloader( batch_size=1, shuffle=False)
dl_val = anss.val.get_dataloader(batch_size=1, shuffle=False)
dl_test = anss.test.get_dataloader(batch_size=1, shuffle=False)

In [7]:
model = eq.models.RecurrentTPP(
    num_extra_features=1, # the number of extra features
    tau_mean=(
        len(anss.train[0])/(anss.train[0].t_end-anss.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [8]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=50, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(model, dl_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type    | Params
------------------------------------------
0 | hypernet_time | Linear  | 3.2 K 
1 | hypernet_mag  | Linear  | 33    
2 | rnn           | GRU     | 3.6 K 
3 | dropout  

Sanity Checking: 0it [00:00, ?it/s]

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
anss_no_depth = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=100,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_depth=False
)

In [ ]:
dl_train = anss_no_depth.train.get_dataloader( batch_size=200, shuffle=False)
dl_val = anss_no_depth.val.get_dataloader(batch_size=200, shuffle=False)
dl_test = anss_no_depth.test.get_dataloader(batch_size=200, shuffle=False)

In [ ]:
model = eq.models.RecurrentTPP(
    num_extra_features=0, # the number of extra features
    tau_mean=(
        len(anss_no_depth.train[0])/(anss_no_depth.train[0].t_end-anss_no_depth.train[0].t_start)
    ), # the mean of the inter-event time distribution
)

In [ ]:
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=50, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=10000, callbacks=[checkpoint, early_stopping],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(model, dl_test)